In [ ]:
import os
import pandas as pd
import numpy as np
import openai
import requests
import pandas as pd
import openai
import json

import logging
import time

from typing import List, Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass

import openrouter



In [ ]:
# Import the openrouter module


# Set the API key
# openrouter.api_key = os.getenv("OPENROUTER_API_KEY")

openrouter.api_key = os.getenv("OPENROUTER_API_KEY")



In [ ]:
disease = 'Aging'

In [ ]:
# Define file path
# file_path_parquet = os.path.join('02_data_processed', '02_covariates_filtered.parquet')

file_path_parquet = os.path.join('02_filtered_data', '02_simple_filter.parquet')

# Load the DataFrame from the Parquet file
filtered_df_covariates = pd.read_parquet(file_path_parquet)

# Display the DataFrame
print(filtered_df_covariates)

In [ ]:
print(filtered_df_covariates.columns)

In [ ]:
import os
import requests
import json
import logging
import pandas as pd
import time
from typing import Dict, List, Optional
from dataclasses import dataclass
from enum import Enum

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class APIError(Exception):
    """Custom exception for API-related errors"""
    pass

class ValidationError(Exception):
    """Custom exception for response validation errors"""
    pass

class BrainStimStatus(str, Enum):
    YES = "Yes"
    NO = "No"

@dataclass
class StimulationParameters:
    frequency: Optional[str] = None
    intensity: Optional[str] = None
    duration: Optional[str] = None

@dataclass
class StimulationDetails:
    primary_type: Optional[str] = None
    is_noninvasive: Optional[bool] = None
    primary_target: Optional[str] = None
    secondary_targets: List[str] = None
    stimulation_parameters: StimulationParameters = None

    def __post_init__(self):
        if self.secondary_targets is None:
            self.secondary_targets = []
        if self.stimulation_parameters is None:
            self.stimulation_parameters = StimulationParameters()

@dataclass
class BrainStimResponse:
    brain_stimulation_used: BrainStimStatus
    stimulation_details: StimulationDetails
    confidence_level: str
    relevant_quotes: List[str]

class BrainStimAnalyzer:
    def __init__(self, api_key: str, site_url: str, app_name: str):
        self.api_key = openrouter.api_key 
        self.site_url = site_url
        self.app_name = app_name
        
        # You can replace or update these models as desired
        self.models = {
            "openai/o1-mini": {
                "max_tokens": 120000,  # manually lowered from 128000
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text.",
                "type": "openai_like"
            },
            "x-ai/grok-2-1212": {
                "max_tokens": 120000,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text.",
                "type": "openai_like"
            },
            "meta-llama/llama-3.3-70b-instruct": {
                "max_tokens": 120000,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text.",
                "type": "openai_like"  # or "llama_like" if needed
            },
            # === Add the new models here ===
            "google/gemini-flash-1.5-8b": {
                "max_tokens": 120000,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text.",
                "type": "openai_like"  # or another type as required by your usage
            },
            "deepseek/deepseek-r1-distill-llama-70b": {
                "max_tokens": 120000,
                "temperature": 0.5,
                "instructions": "Return ONLY a JSON object with no additional explanatory text.",
                "type": "openai_like"
            }
        }

    def extract_json_from_text(self, text: str) -> Optional[str]:
        """
        Attempt to extract a JSON substring from a block of text.
        Return None if not valid JSON or cannot find curly braces.
        """
        if not text or not isinstance(text, str):
            return None

        try:
            start = text.find('{')
            end = text.rfind('}')
            if start != -1 and end != -1:
                json_str = text[start:end+1]
                # Validate it's proper JSON
                json.loads(json_str)
                return json_str
        except json.JSONDecodeError:
            pass
        return None

    def validate_brain_stim_response(self, response_json: Dict) -> BrainStimResponse:
        """Validate response structure and content, handling any None fields gracefully."""
        required_fields = [
            "brain_stimulation_used",
            "stimulation_details",
            "confidence_level",
            "relevant_quotes"
        ]

        for field in required_fields:
            if field not in response_json:
                raise ValidationError(f"Missing required field: {field}")

        # Validate brain_stimulation_used
        if not isinstance(response_json["brain_stimulation_used"], str):
            raise ValidationError("brain_stimulation_used must be string")
        if response_json["brain_stimulation_used"] not in ["Yes", "No"]:
            raise ValidationError("brain_stimulation_used must be 'Yes' or 'No'")

        # Safely handle any missing or null "stimulation_details"
        stim_details_data = response_json.get("stimulation_details") or {}
        # Safely handle any missing or null "stimulation_parameters"
        stim_params_data = stim_details_data.get("stimulation_parameters") or {}

        # Build StimulationParameters object
        stim_params = StimulationParameters(**stim_params_data)

        # Build StimulationDetails object
        stim_details = StimulationDetails(
            primary_type=stim_details_data.get("primary_type"),
            is_noninvasive=stim_details_data.get("is_noninvasive"),
            primary_target=stim_details_data.get("primary_target"),
            secondary_targets=stim_details_data.get("secondary_targets"),
            stimulation_parameters=stim_params
        )

        # Make sure relevant_quotes is a list
        relevant_quotes = response_json.get("relevant_quotes")
        if not isinstance(relevant_quotes, list):
            relevant_quotes = []

        return BrainStimResponse(
            brain_stimulation_used=BrainStimStatus(response_json["brain_stimulation_used"]),
            stimulation_details=stim_details,
            confidence_level=response_json["confidence_level"],
            relevant_quotes=relevant_quotes
        )

    def send_request(self, model_name: str, prompt: str, max_retries: int = 3) -> Optional[BrainStimResponse]:
        """
        Send request to OpenRouter with retry logic.
        Returns a BrainStimResponse or None if there's an error or invalid response.
        """
        if model_name not in self.models:
            raise ValueError(f"Unknown model: {model_name}")

        model_config = self.models[model_name]

        # We'll attach our special instructions plus the user prompt
        final_prompt = f"{model_config['instructions']}\n{prompt}"

        json_body = {
            "model": model_name,
            "temperature": model_config["temperature"],
            "max_tokens": model_config["max_tokens"],
            # OpenAI-like messages
            "messages": [{"role": "user", "content": final_prompt}]
        }

        retries = 0
        backoff = 2  # seconds

        while retries < max_retries:
            try:
                response = requests.post(
                    url="https://openrouter.ai/api/v1/chat/completions",
                    headers={
                        "Authorization": f"Bearer {self.api_key}",
                        "HTTP-Referer": self.site_url,
                        "X-Title": self.app_name,
                        "Content-Type": "application/json"
                    },
                    data=json.dumps(json_body),
                    timeout=60
                )
                logging.info(f"Sent request to {model_name}")
                logging.debug(f"Raw response text: {response.text}")

                # Attempt to parse the top-level JSON
                try:
                    response_data = response.json()
                except json.JSONDecodeError:
                    logging.error(f"Response from {model_name} is not valid JSON: {response.text}")
                    return None

                # Check if top-level JSON indicates an error
                if "error" in response_data:
                    logging.error(f"Model {model_name} returned an error: {response_data['error']}")
                    return None

                # Now handle different status codes
                if response.status_code == 200:
                    # For an OpenAI-like response, read from "choices"
                    choices = response_data.get("choices", [])
                    if not choices:
                        logging.error("Response does not contain 'choices' or it's empty.")
                        return None

                    choice = choices[0]
                    message_obj = choice.get("message")
                    if not message_obj or "content" not in message_obj:
                        logging.error("Response does not contain 'message' or 'content' keys.")
                        return None

                    content = message_obj["content"]
                    # Safely handle None or empty content
                    if not content or not isinstance(content, str):
                        logging.error("Content is empty or not a string.")
                        return None

                    content = content.strip()
                    # Attempt to parse the content as JSON
                    try:
                        response_json = json.loads(content)
                    except json.JSONDecodeError:
                        # Try to extract JSON substring if the response has extra text
                        json_str = self.extract_json_from_text(content)
                        if not json_str:
                            logging.error(f"Could not extract valid JSON from response: {content}")
                            return None
                        try:
                            response_json = json.loads(json_str)
                        except json.JSONDecodeError:
                            logging.error(f"Could not parse JSON substring from response: {content}")
                            return None

                    # Validate final JSON structure
                    try:
                        return self.validate_brain_stim_response(response_json)
                    except ValidationError as ve:
                        logging.error(f"Validation Error: {str(ve)}")
                        return None

                elif response.status_code in [429, 500, 502, 503, 504]:
                    # Retry for these statuses
                    if retries == max_retries - 1:
                        logging.error(f"Max retries reached for {model_name} with status {response.status_code}")
                        return None
                    retries += 1
                    time.sleep(backoff)
                    backoff *= 2
                    continue
                else:
                    # Other statuses: treat as non-retryable
                    logging.error(f"Request failed with status {response.status_code}: {response.text}")
                    return None

            except requests.exceptions.RequestException as e:
                # This covers any network issues, timeouts, etc.
                logging.error(f"Request to {model_name} failed: {e}")
                if retries == max_retries - 1:
                    return None
                retries += 1
                time.sleep(backoff)
                backoff *= 2
                continue

        return None

class BrainStimAnalysis:
    def __init__(self, api_key: str, site_url: str, app_name: str):
        self.analyzer = BrainStimAnalyzer(api_key, site_url, app_name)

    def process_trial(self, df: pd.DataFrame, output_dir: str = '03_data_ai'):
        """Process trial data and save results."""
        os.makedirs(output_dir, exist_ok=True)
        result_df = df.copy()

        # Initialize response arrays
        responses = []
        
        # Process each row
        for idx, row in df.iterrows():
            # Combine your instructions with the row text
            context = row.get('DetailedDescription') or row.get('BriefSummary') or ""
            prompt = self.generate_prompt(context)

            # For each model in the dictionary
            for model_name in self.analyzer.models.keys():
                response = self.analyzer.send_request(model_name, prompt)
                if response is None:
                    logging.error(f"Error processing row {idx} with model {model_name}.")
                    # Skip this model and continue to the next
                    continue

                # Store the valid response
                responses.append({"Model": model_name, "Response": response})
                
                # Update DataFrame with response data
                self.update_dataframe(result_df, idx, model_name, response)

        # Save results
        self.save_results(result_df, output_dir)
        return result_df, responses

    def generate_prompt(self, context: str) -> str:
        """Generate analysis prompt from context."""
        return f"""
        Analyze whether brain stimulation was used in this trial. If so, provide details.

        IMPORTANT: Respond ONLY with a JSON object in the EXACT format below. Do not include any additional text or explanations.

        {{
            "brain_stimulation_used": "Yes" or "No",
            "stimulation_details": {{
                "primary_type": "e.g., tDCS, TMS, tACS, DBS, etc." or null,
                "is_noninvasive": true or false,
                "primary_target": "Primary brain region or null",
                "secondary_targets": ["List of secondary regions"] or [],
                "stimulation_parameters": {{
                    "frequency": "e.g., 10Hz" or null,
                    "intensity": "e.g., 2mA" or null,
                    "duration": "e.g., 20 minutes" or null
                }}
            }},
            "confidence_level": "High", "Medium", or "Low",
            "relevant_quotes": ["Direct quotes supporting the analysis"]
        }}

        Context:
        {context}
        """

    def update_dataframe(self, df: pd.DataFrame, idx: int, model_name: str, response: BrainStimResponse):
        """Update DataFrame with response data."""
        suffix = model_name.replace("/", "_").replace("-", "_")
        
        # Update brain stimulation status
        df.at[idx, f"brain_stimulation_used_{suffix}"] = response.brain_stimulation_used.value
        
        # Update stimulation details
        if response.stimulation_details:
            df.at[idx, f"stimulation_details_primary_type_{suffix}"] = response.stimulation_details.primary_type
            df.at[idx, f"stimulation_details_is_noninvasive_{suffix}"] = response.stimulation_details.is_noninvasive
            df.at[idx, f"stimulation_details_primary_target_{suffix}"] = response.stimulation_details.primary_target
            df.at[idx, f"stimulation_details_secondary_targets_{suffix}"] = json.dumps(response.stimulation_details.secondary_targets)
            
            # Update parameters
            params = response.stimulation_details.stimulation_parameters
            df.at[idx, f"stimulation_details_parameters_frequency_{suffix}"] = params.frequency
            df.at[idx, f"stimulation_details_parameters_intensity_{suffix}"] = params.intensity
            df.at[idx, f"stimulation_details_parameters_duration_{suffix}"] = params.duration
        
        # Update confidence and quotes
        df.at[idx, f"confidence_level_{suffix}"] = response.confidence_level
        df.at[idx, f"relevant_quotes_{suffix}"] = json.dumps(response.relevant_quotes)

    def save_results(self, df: pd.DataFrame, output_dir: str):
        """Save results to files."""
        df.to_excel(f'{output_dir}/part_3_covariates_done.xlsx', index=False)
        df.to_parquet(f'{output_dir}/part_3_covariates_done.parquet', index=False)

def main():
    # Example config - use your actual approach to load the API key
    api_key = os.getenv("OPENROUTER_API_KEY")
    site_url = 'https://Aged-Research.com'
    app_name = 'Aged + NIBS'

    if not api_key:
        logging.error("API_KEY is not set. Please set the OPENROUTER_API_KEY environment variable.")
        return

    try:
        # Load your DataFrame however you do (Excel, CSV, etc.)
        # df = pd.read_excel('filtered_df_covariates.xlsx')
        df = filtered_df_covariates  # Replace with your real DataFrame variable
        df = df.reset_index(drop=True)

        # Run analysis
        analysis = BrainStimAnalysis(api_key, site_url, app_name)
        result_df, responses = analysis.process_trial(df)

        # Save additional outputs
        output_dir = '03_data_ai'
        os.makedirs(output_dir, exist_ok=True)

        result_df.to_excel(f'{output_dir}/part_3_covariates_done_v2.xlsx', index=False)
        result_df.to_parquet(f'{output_dir}/part_3_covariates_done_v2.parquet', index=False)

        # Convert responses to DataFrame and save
        responses_df = pd.DataFrame([
            {
                "Model": r["Model"],
                "BrainStimulationUsed": r["Response"].brain_stimulation_used.value,
                "PrimaryType": r["Response"].stimulation_details.primary_type,
                "ConfidenceLevel": r["Response"].confidence_level,
                "Quotes": r["Response"].relevant_quotes
            }
            for r in responses
        ])
        responses_df.to_excel(f'{output_dir}/responses_done_v2.xlsx', index=False)
        responses_df.to_parquet(f'{output_dir}/responses_done_v2.parquet', index=False)

        logging.info("All results saved successfully.")

        # Optional: Display results
        print("\n=== Updated LLM Responses ===")
        print(result_df)

    except Exception as e:
        logging.error(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()


In [ ]:
import requests



resp = requests.post('https://textbelt.com/text', {

  'phone': '9163802941',

  'message': 'PD Pipe Line Part 3 Done',

  'key': '138adc496234ca311154757db147f552afa8ba83FfrCKJ36kTJNXq65nlsvvF4Pu',

})

print(resp.json())